In [1]:
# NS
#optiver-ver (for ensemble)

#optiver-ver-nn-modeling/inference-v1
#optiver-ver-rnn-modeling/inference-v1
#optiver-ver-lgb-modeling/inference-v1


import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

from scipy.stats import hmean

from sklearn.ensemble import HistGradientBoostingRegressor
import itertools
import pickle
import joblib
from itertools import combinations
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate,Dropout
import pickle
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Add

#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

jointquant          = True
is_inference        = False
load_models         = False
run_pipeline        = True
train_models        = True
is_rnn              = True   
online_learning     = True




#public-validation
dates_train = ["2020-06-01","2021-02-29"]
dates_test = ["2021-03-01","2021-06-30"]

#full-inference
#dates_train = [0,480]
#dates_test = [-1,-1]


num_models ={'rnn':2}


if jointquant:
    train_2018_1000_path = "A:\joinquant/factor/1000/2018_1000.csv.gz"
    train_2018_1330_path = 'A:\joinquant/factor/1330/2018_1330.csv.gz'
    train_2019_1000_path = "A:\joinquant/factor/1000/2019_1000.csv.gz"
    train_2019_1330_path = 'A:\joinquant/factor/1330/2019_1330.csv.gz'
    train_2020_1000_path = "A:\joinquant/factor/1000/2020_1000.csv.gz"
    train_2020_1330_path = 'A:\joinquant/factor/1330/2020_1330.csv.gz'
    train_2021_1000_path = "A:\joinquant/factor/1000/2021_1000.csv.gz"
    train_2021_1330_path = 'A:\joinquant/factor/1330/2021_1330.csv.gz'
    train_2022_1000_path = "A:\joinquant/factor/1000/2022_1000.csv.gz"
    train_2022_1330_path = 'A:\joinquant/factor/1330/2022_1330.csv.gz'
    train_2023_1000_path = "A:\joinquant/factor/1000/2023_1000.csv.gz"
    train_2023_1330_path = 'A:\joinquant/factor/1330/2023_1330.csv.gz'
    train_path = 'A:\joinquant/train.csv'
    models_path = "A:\joinquant\model"
else:
    models_path = "A:\joinquant\model"
    train_path = "A:\joinquant/factor/1000/2018_1000.csv.gz"
    train_path2 = 'A:\joinquant/factor/1330/2018_1330.csv.gz'
    

if dates_train[1]!="2023-06-30":
    models_path = "A:\joinquant/vals-258"
#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

simulation_path = "vals-258/"

In [2]:
rnn_ep         = 1000
rnn_lr         = 0.001
rnn_bs         = 2**12
window_size    = 2

In [3]:
def read_and_concatenate(year):
    # 文件路径
    path_1000 = f"A:\\joinquant/factor/1000/{year}_1000.csv.gz"
    path_1330 = f"A:\\joinquant/factor/1330/{year}_1330.csv.gz"
    
    # 读取数据
    df_1000 = pd.read_csv(path_1000, compression='gzip')
    df_1330 = pd.read_csv(path_1330, compression='gzip')

    # 为每个DataFrame添加时间列
    df_1000['time'] = '10:00:00'
    df_1330['time'] = '13:30:00'
    
    # 使用concat而不是merge
    concatenated_df = pd.concat([df_1000, df_1330], axis=0)
    
    # 可以选择排序，如果需要按照日期和代码顺序排列
    concatenated_df.sort_values(by=['date', 'code', 'time'], inplace=True)

    return concatenated_df

# 处理每一年的数据
years = [2020,2021]
all_data = pd.concat([read_and_concatenate(year) for year in years], axis=0)

# all_data 现在包含了所有年份的数据，每个时间点的数据并排放置

path_lable = "A:\joinquant/label.csv.gz"
lable1 = pd.read_csv(path_lable, compression = "gzip") 
# 合并 all_data 和 label DataFrame
train = pd.merge(all_data, lable1, on=['date', 'code',"time"], how='inner')

# merged_df 现在包含了合并后的数据
del lable1 , all_data

In [4]:
train = train.drop('ret_next_5_close_alpha', axis=1)

In [5]:
def convert_price_cols_float32(df):

    # all Columns 
    all_columns = [col for col in df.columns if 'factor' in col]
    df[all_columns] = df[all_columns].astype('float32')

    return df

#train.to_csv('A:\joinquant/train.csv', index=False)

In [6]:
import pandas as pd



# 按照股票ID和日期进行分组，并计算每组中的唯一时间点数量
grouped = train.groupby(['date', 'code'])['time'].nunique()

# 找出只有一个时间点的组（即缺失另一个时间点的情况）
missing_one_period = grouped[grouped == 1]

# 获取缺失一个时间点的组合的索引
missing_index = missing_one_period.index

# 使用索引剔除这些组
filtered_train = train.set_index(['date', 'code'])
filtered_train = filtered_train.drop(index=missing_index)

# 重置索引
filtered_train = filtered_train.reset_index()

# 计算剔除缺失数据后的唯一天数
unique_days_after_filtering = filtered_train['date'].nunique()
print(f"剔除缺失数据后的唯一天数: {unique_days_after_filtering}")
#filtered_train.to_csv('A:\joinquant/train.csv', index=False)

剔除缺失数据后的唯一天数: 486


In [7]:
train = filtered_train
del filtered_train

In [8]:
#train = pd.read_csv(train_path)

print(f"converting column names")
train = train.rename(columns = {'ret_next_close_alpha':'target','code':'stock_id','date':'date_id'})

nan_count = train['target'].isna().sum()
print(f"The 'target' column has {nan_count} NaN values.")
target_median = train['target'].median()
train['target'].fillna(target_median, inplace=True)

print(f"converting prices columns to float32 values.")
train = convert_price_cols_float32(train)

print(f"transforming datetime into float")
time_replace_dict = {
    '13:30:00': 13.5,
    '10:00:00': 10
    }
train['time'] = train['time'].replace(time_replace_dict)
print("done")
# ----------------------------- Reading train data -------------------------

converting column names
The 'target' column has 0 NaN values.
converting prices columns to float32 values.
transforming datetime into float
done


In [9]:
train

,date_id,stock_id,factor000,factor001,factor002,factor003,factor004,factor005,factor006,factor007,...,factor492,factor493,factor494,factor495,factor496,factor497,factor498,factor499,time,target
0,2020-01-02,000001.XSHE,1.6170,1.5625,0.59670,0.641000,-0.9060,-0.79740,0.3887,-0.6290,...,-1.6690,1.5910,0.6390,0.05856,0.9995,-1.1820,0.4705,-0.252400,10.0,0.017486
1,2020-01-02,000001.XSHE,1.1840,0.9710,0.05646,1.032000,-0.8530,-0.98970,-1.1440,1.1800,...,-1.6580,-0.4045,-0.9010,-0.11444,1.1850,-0.3271,-1.5410,-1.129000,13.5,0.014280
2,2020-01-02,000002.XSHE,-0.1251,1.3260,-1.45300,-0.012115,-0.3140,-0.06256,0.9420,-0.0545,...,-1.6990,0.8623,-0.1181,-1.37600,-1.0460,-0.8540,-0.6753,-0.822800,10.0,-0.050853
3,2020-01-02,000002.XSHE,0.5874,0.2568,-1.50800,1.003000,0.2305,-1.35450,1.4990,0.1063,...,-1.6800,-0.1908,-0.8340,-1.41500,-0.8040,-0.0453,-1.0090,-0.796400,13.5,-0.031760
4,2020-01-02,000004.XSHE,-0.3320,-1.0400,-1.68600,-0.871000,1.5510,1.65400,-0.2201,-0.7217,...,-1.0530,-1.2110,0.1969,1.56900,-0.0424,-1.3630,0.5920,0.878400,10.0,-0.027291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409703,2021-12-31,688788.XSHG,0.6055,-1.2730,-1.02050,1.124000,1.1820,1.24800,0.4949,-0.7260,...,1.6970,-1.4590,-1.6720,1.62400,0.2788,-1.2740,0.2078,1.092000,13.5,0.003397
3409704,2021-12-31,688981.XSHG,1.6090,0.7280,-1.59000,-1.527000,1.5220,1.45700,1.5400,-1.5310,...,1.3550,-0.1527,-0.3150,0.85840,0.2568,-1.5840,1.2800,-0.583000,10.0,-0.009358
3409705,2021-12-31,688981.XSHG,1.6840,-0.2489,-1.11500,-1.232000,1.6760,0.88130,1.6720,-1.7190,...,0.5960,0.1161,1.1270,1.51100,-1.0850,-1.6410,1.5480,1.349000,13.5,-0.008405
3409706,2021-12-31,689009.XSHG,0.3218,-0.2747,-1.02100,-0.064000,1.7230,1.27400,1.6010,-1.7270,...,0.2397,0.5796,-1.5170,0.21600,-1.2560,-1.6875,1.1540,-1.397000,10.0,-0.001219


In [10]:
#@title functions

def split_by_date(df, dates):

    df_start, df_end = dates
    df = df[(df['date_id'] >= df_start) & (df['date_id'] <=df_end)].reset_index(drop=True)

    return df



def lag_function(df, columns_to_lag, numbers_of_days_to_lag):

    df_indexed = df.set_index(['stock_id', 'time', 'date_id'])

    for column_to_lag in columns_to_lag:
        for number_days_to_lag in numbers_of_days_to_lag:
            df_indexed[f'lag{number_days_to_lag}_{column_to_lag}'] = df_indexed.groupby(level=['stock_id', 'time'])[column_to_lag].shift(number_days_to_lag)

    df_indexed.reset_index(inplace=True)

    return df_indexed




def compute_imbalances(df_, columns, prefix = ''):
    """Computes the differences and imbalances for pairs of columns and stores them in the DataFrame."""
    df = df_.copy()
    for col1, col2 in combinations(columns, 2):

        # Sort the columns lexicographically to ensure consistent ordering
        col1, col2 = sorted([col1, col2])

        # Compute imbalance directly without creating a temporary difference column
        total = df[col1] + df[col2]
        imbalance_column_name = f'{col1}_{col2}_imb{prefix}'

        # Ensure we don't divide by zero
        df[imbalance_column_name] = (df[col1] - df[col2]).divide(total, fill_value=np.nan)

    return df



def save_pickle(data, file_path):

    # Create the directory if it doesn't exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the pickle file
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

    print(f"Data saved to {file_path}")
    #example: save_pickle(all_data, 'k8/all_data.pkl')

def load_pickle(file_path):

    # Load and return the data from the pickle file
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        return data
    else:
        raise FileNotFoundError(f"No such file: {file_path}")

    #example: loaded_data = load_pickle('k8/all_data.pkl')

In [1]:
pip install xelatex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xelatex (from versions: none)
ERROR: No matching distribution found for xelatex


In [11]:
num_of_target_lags  = 3
target_lags         = list(range(1,num_of_target_lags+1))


def feature_pipeline(df):

    if df.empty:
        return pd.DataFrame()


    # df = compute_imbalances(df, columns_sizes,prefix='_sz_')
    # df = compute_imbalances(df, columns_prices,prefix = '_pr_')


    print(f"lagging target column for {len(target_lags)} lags.")
    df = lag_function(df, ['target'], target_lags)



    print("Done...")

    return df

In [12]:
def make_predictions(models, X_test,model = 'nn'):
    if model == 'nn':
        all_predictions = [model.predict(X_test, batch_size=16384) for model in models]
    if model == 'lgb' or model == 'xgb' or model == 'cat':
        all_predictions = [model.predict(X_test) for model in models]
    prediction = np.mean(all_predictions, axis=0)
    return prediction

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

class BestScoresCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_train_loss = float('inf')
        self.best_val_loss = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss', float('inf'))
        val_loss = logs.get('val_loss', float('inf'))

        if train_loss < self.best_train_loss:
            self.best_train_loss = train_loss
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss

    def on_train_end(self, logs=None):
        print(f"Best training loss: {self.best_train_loss}, Best validation loss: {self.best_val_loss}")

In [13]:
#@title RNN second pass

def precompute_sequences(stock_data, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num = stock_data[rnn_numerical_features].values
    stock_data_cat = stock_data[rnn_categorical_features].values

    # Pre-compute all sequences
    all_sequences_num = [stock_data_num[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]
    all_sequences_cat = [stock_data_cat[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]

    # Add padding if necessary
    padded_sequences_num = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_num]
    padded_sequences_cat = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_cat]

    # Combine numerical and categorical features
    combined_sequences = np.array([np.concatenate([num, cat], axis=-1)
                                   for num, cat in zip(padded_sequences_num, padded_sequences_cat)])

    # Extract targets
    targets = stock_data['target'].values

    return combined_sequences, targets

def get_sequence(precomputed_data, time_step):
    combined_sequences, targets = precomputed_data
    return combined_sequences[time_step], targets[time_step]



def create_batches(data, window_size, rnn_numerical_features, rnn_categorical_features, max_time_steps= 2):

    grouped = data.groupby(['stock_id', 'date_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(max_time_steps):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    return all_batches, all_targets




def compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num      = group[rnn_numerical_features].values
    stock_data_cat      = group[rnn_categorical_features].values
    stock_data_target   = group['target'].values

    # Find the index of the target second
    target_index = len(group) - 1

    # Extract the sequence for the target index
    sequence_num = stock_data_num[max(0, target_index - window_size + 1):target_index + 1]
    sequence_cat = stock_data_cat[max(0, target_index - window_size + 1):target_index + 1]

    # Add padding if necessary
    padded_sequence_num = np.pad(sequence_num, ((window_size - len(sequence_num), 0), (0, 0)), 'constant')
    padded_sequence_cat = np.pad(sequence_cat, ((window_size - len(sequence_cat), 0), (0, 0)), 'constant')

    # Combine numerical and categorical features
    combined_sequence = np.concatenate([padded_sequence_num, padded_sequence_cat], axis=-1)

    # Extract target
    target = stock_data_target[-1]

    return combined_sequence, target


def create_last_batches(data, window_size, rnn_numerical_features, rnn_categorical_features):

    grouped = data.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in grouped:
        # Compute the sequence for the last data point in the current group
        last_sequence, last_target = compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Check if the sequence is valid (i.e., not empty)
        if last_sequence.size > 0:
            all_batches.append(last_sequence)
            all_targets.append(last_target)

    return all_batches, all_targets





def second_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=False):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    if is_inference:
        df_copy.fillna(rnn_medians, inplace=True)
        df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])



    if is_inference:
        df_copy_batches, df_copy_targets = create_last_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)
    else:
        df_copy_batches, df_copy_targets = create_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)


    return df_copy_batches, df_copy_targets


def online_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])


    grouped = df_copy.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(2):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    df_batches = np.array(all_batches)
    df_targets = np.array(all_targets)


    return df_batches, df_targets

In [14]:
#@title RNN model

def apply_conv_layers(input_layer, kernel_sizes, filters= 16, do_ratio=0.5):
    conv_outputs = []

    for kernel_size in kernel_sizes:
        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(input_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Dropout(do_ratio)(conv_layer)

        shortcut = conv_layer

        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(conv_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Activation('relu')(conv_layer)

        # Add the output of the first Conv1D layer
        conv_layer = Add()([conv_layer, shortcut])
        conv_outputs.append(conv_layer)


    concatenated_conv = Concatenate(axis=-1)(conv_outputs)
    flattened_conv_output = Flatten()(concatenated_conv)

    return flattened_conv_output

def create_rnn_model_with_residual(window_size, numerical_features, initial_learning_rate=0.001):

    categorical_features = 'time'
    categorical_uniques  = {'time' : 2}
    embedding_dim        = {'time' : 10}

    input_layer = Input(shape=(window_size, len(numerical_features) + 1), name="combined_input")

    # Split the input into numerical and categorical parts
    numerical_input = Lambda(lambda x: x[:, :, :-1], name="numerical_part")(input_layer)
    categorical_input = Lambda(lambda x: x[:, :, -1:], name="categorical_part")(input_layer)

    first_numerical = Lambda(lambda x: x[:, 0])(numerical_input)


    # diffrentiate layers
    def create_difference_layer(lag):
        return Lambda(lambda x: x[:, lag:, :] - x[:, :-lag, :], name=f"difference_layer_lag{lag}")

    difference_layers = []
    for lag in range(1, window_size):
        diff_layer = create_difference_layer(lag)(numerical_input)
        padding = ZeroPadding1D(padding=(lag, 0))(diff_layer)  # Add padding to the beginning of the sequence
        difference_layers.append(padding)
    combined_diff_layer = Concatenate(name="combined_difference_layer")(difference_layers)


    # Embedding for categorical part
    vocab_size, embedding_dim = categorical_uniques[categorical_features], embedding_dim[categorical_features]
    embedding = Embedding(vocab_size, embedding_dim, input_length=window_size)(categorical_input)
    embedding = Reshape((window_size, -1))(embedding)

    first_embedding = Lambda(lambda x: x[:, 0])(embedding)

    # Concatenate numerical input and embedding
    # conv_input = concatenate([enhanced_numerical_input, embedding], axis=-1)

    kernel_sizes = [2,3]
    do_ratio = 0.4

    flattened_conv_output = apply_conv_layers(numerical_input, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_cat = apply_conv_layers(embedding, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_diff = apply_conv_layers(combined_diff_layer, kernel_sizes, do_ratio=do_ratio)


    dense_output = Concatenate(axis=-1)([flattened_conv_output,flattened_conv_output_cat,flattened_conv_output_diff, Reshape((-1,))(combined_diff_layer),first_numerical,first_embedding])

    dense_sizes = [512, 256, 128, 64, 32]
    do_ratio = 0.5
    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(do_ratio)(dense_output)

    # Output layer
    output = Dense(1, name='output_layer')(dense_output)

    # Learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.7,
        staircase=True)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss="mean_absolute_error")

    return model

In [15]:
#@title runnig pipeline

excluded_columns = ['date_id','target']

if run_pipeline:

    train_eng = feature_pipeline(train)

    if is_rnn:
        excluded_columns = excluded_columns + ['stock_id']

        features = [col for col in train_eng.columns if col not in excluded_columns]
        rnn_categorical_features =  ['time']
        rnn_numerical_features = [feat for feat in features if feat not in rnn_categorical_features]
        print("we have {} numerical and {} categorical".format(len(rnn_numerical_features),len(rnn_categorical_features)))


        rnn_scaler = StandardScaler()
        rnn_medians = train_eng.median(numeric_only = True)

        train_eng.fillna(rnn_medians, inplace=True)
        train_eng[rnn_numerical_features] = rnn_scaler.fit_transform(train_eng[rnn_numerical_features])


        rnn_all_data = {
            "rnn_scaler": rnn_scaler,
            "rnn_medians": rnn_medians,
            "rnn_categorical_features": rnn_categorical_features,
            "rnn_numerical_features": rnn_numerical_features
        }

        save_pickle(rnn_all_data, f'{models_path}rnn_all_data.pkl')
        print("Pipline Done!")
    cleaning = True
    if cleaning:
        import gc
        del train
    print("splitting train data")
    train_data       = split_by_date(train_eng, dates_train)
    print("splitting test data")
    test_data        = split_by_date(train_eng, dates_test)
    print("number of dates in train = {} , number of dates in test {}".format (train_data['date_id'].nunique(),test_data['date_id'].nunique()))

    cleaning = True
    if cleaning:
        import gc
        #del train
        del train_eng
        gc.collect()

lagging target column for 3 lags.
Done...
we have 503 numerical and 1 categorical
Data saved to A:\joinquant/vals-258rnn_all_data.pkl
Pipline Done!
splitting train data
splitting test data
number of dates in train = 181 , number of dates in test 83


In [16]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import tensorflow.keras.backend as K
print('Using TensorFlow version',tf.__version__)

# RESTRICT TENSORFLOW TO 8GB OF GPU RAM
# SO THAT WE HAVE 8GB RAM FOR RAPIDS
LIMIT = 8
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)



Using TensorFlow version 2.10.1
We will restrict TensorFlow to max 8GB GPU RAM


In [17]:
#@title prepare train models
if train_models:

    if test_data.empty:
        print("--------------test data is empty so adjusting the last date for test-----------------")
        test_data = train_data.query("date_id == 2021-12-31").copy()

    if is_rnn:

        train_batches, train_targets = second_pass_for_rnn(train_data, rnn_numerical_features, rnn_categorical_features, window_size)
        del train_data
        test_batches, test_targets  = second_pass_for_rnn(test_data, rnn_numerical_features, rnn_categorical_features, window_size)
        del test_data
        print(f"train batches shape:{train_batches.shape}")

Processing groups: 100%|█████████████████████████████████████████████████████| 294631/294631 [03:39<00:00, 1342.60it/s]


train batches shape:(1232932, 2, 504)


In [18]:
#@title training models
# train_data = flatten_outliers(train_data, 'target', lower_quantile=0.01, upper_quantile=0.99)
if train_models:

        directory = os.path.dirname(models_path)
        if not os.path.exists(directory):
            os.makedirs(directory)


        if is_rnn:

            callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
            if dates_train[1] != "2021-12-31":
                early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
                callbacks.append(early_stopping)

            if True or tpu_strategy:
                # with tpu_strategy.scope():
                    rnn_models = []
                    for i in range(num_models['rnn']):
                        #resetgpu()
                        print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
                        print("---------------------------------------")
                        set_all_seeds(42+i)

                        rnn_model = create_rnn_model_with_residual(window_size, rnn_numerical_features, initial_learning_rate=rnn_lr)
                        history = rnn_model.fit(train_batches, train_targets, validation_data=(test_batches, test_targets), epochs=rnn_ep, batch_size=rnn_bs, callbacks=callbacks)
                        print("---------------------------------------")
                        rnn_model.save(f'{models_path}rnn_model_seed_{i+1}.h5')
                        rnn_models.append(rnn_model)
                     
                    predictions =  make_predictions(rnn_models, test_batches,model = 'nn')
                    print(f"Ensemble Mean Absolute Error: {mean_absolute_error(test_targets, predictions):.4f}")

Training rnn model 1 out of 2 with seed 42
---------------------------------------
Epoch 1/1000
302/302 [==============================] - 20s 37ms/step - loss: 0.2039 - val_loss: 0.0355
Epoch 2/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0403 - val_loss: 0.0208
Epoch 3/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0317 - val_loss: 0.0210
Epoch 4/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0280 - val_loss: 0.0195
Epoch 5/1000
302/302 [==============================] - 9s 31ms/step - loss: 0.0237 - val_loss: 0.0196
Epoch 6/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0208 - val_loss: 0.0192
Epoch 7/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0201 - val_loss: 0.0196
Epoch 8/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0203 - val_loss: 0.0195
Epoch 9/1000
302/302 [==============================] - 9s 30ms/step - loss: 0.0202 - val_lo

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import tensorflow.keras.backend as K
print('Using TensorFlow version',tf.__version__)

# RESTRICT TENSORFLOW TO 8GB OF GPU RAM
# SO THAT WE HAVE 8GB RAM FOR RAPIDS
LIMIT = 8
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)

print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
print("---------------------------------------")
set_all_seeds(42+i)

rnn_model = create_rnn_model_with_residual(window_size, rnn_numerical_features, initial_learning_rate=rnn_lr)
history = rnn_model.fit(train_batches, train_targets, validation_data=(test_batches, test_targets), epochs=rnn_ep, batch_size=rnn_bs, callbacks=callbacks)
print("---------------------------------------")
rnn_model.save(f'{models_path}rnn_model_seed_{i+1}.h5')
rnn_models.append(rnn_model)

Using TensorFlow version 2.10.1
We will restrict TensorFlow to max 8GB GPU RAM
Training rnn model 2 out of 2 with seed 43
---------------------------------------
